In [29]:
import requests

from ape import networks, Contract
import pandas as pd
from pycoingecko import CoinGeckoAPI

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [31]:
prices = CoinGeckoAPI().get_price(["aura-finance", "balancer", "badger-dao"], "usd")
bal_price = prices["balancer"]["usd"]
aura_price = prices["aura-finance"]["usd"]
badger_price = prices["badger-dao"]["usd"]
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price)]]
headers = ["Badger price", "Balancer price", "Aura price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price
0,$3.65,$5.29,$1.84


In [32]:
# endpoint and query for aura proposals
SNAPSHOT_SUBGRAPH = "https://hub.snapshot.org/graphql?"
PROPOSAL_INFO_QUERY = """
        query($proposal_id: String) {
          proposal(id: $proposal_id) {
            scores_total
            snapshot
            choices
            scores
          }
        }
        """
proposal_id_last = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"
proposal_id_current = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"

# endpoint for llama for cost per vote last round
LLAMA_DASHBOARD_URL = "https://api.llama.airforce//dashboard"
aura_dash = requests.post(LLAMA_DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
cost_per_vote_last_round = aura_dash['dashboard']['epochs'][-1]['dollarPerVlAsset']
cost_per_vote_after_fee = cost_per_vote_last_round * (1.04)

# check active voter in past proposal as ref
response_proposal = requests.post(
        SNAPSHOT_SUBGRAPH,
        json={"query": PROPOSAL_INFO_QUERY, "variables": {"proposal_id": proposal_id_last}},
    ).json()["data"]["proposal"]
response_proposal = requests.post(
        SNAPSHOT_SUBGRAPH,
        json={"query": PROPOSAL_INFO_QUERY, "variables": {"proposal_id": proposal_id_current}},
    ).json()["data"]["proposal"]

block_current_proposal = int(response_proposal["snapshot"])
aura_voting_actively_last_round = response_proposal["scores_total"]
data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', "${:.4f}".format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,15635887,6830592,$0.0660


In [33]:
# emissions ecosystem
BALANCER_EMISSIONS = 145_000
AURA_FEE = .25
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
total_cliffs = aura.totalCliffs()
cliff_reduction = aura.reductionPerCliff() / 1e18
aura_total_supply = aura.totalSupply(block_identifier=block_current_proposal) / 1e18
init_mint_amount = aura.INIT_MINT_AMOUNT() / 1e18
aura_mint_ratio = (
    (total_cliffs - (aura_total_supply - init_mint_amount) / cliff_reduction) * 2.5
    + 700
) / total_cliffs
weekly_emissions = BALANCER_EMISSIONS * bal_price + BALANCER_EMISSIONS * aura_mint_ratio * aura_price
weekly_emissions_after_fee = weekly_emissions * (1-AURA_FEE)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, "${:.4f}".format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions


,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.752714,$2652411.0002


In [34]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
strat_votes = vlAURA.getVotes('0x14f83ff95d4ec5e8812ddf42da1232b0ba1015e6', block_identifier=block_current_proposal) /1e18
council_fee = strat_votes*.1
treasury_votes = vlAURA.getVotes('0xA9ed98B5Fb8428d68664f3C5027c62A10d45826b', block_identifier=block_current_proposal) /1e18
total_vp_badgerdao = treasury_votes + council_fee
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
badger_aura_control = "{0:.3%}".format(pct_controlled_aura_by_badger)
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
badger_aura_control_active = "{0:.3%}".format(pct_controlled_aura_by_badger_active)
data = [[treasury_votes, council_fee, total_vp_badgerdao, badger_aura_control, badger_aura_control_active]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,182298.367156,91833.486207,274131.853364,3.396%,4.013%


In [35]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_aura_vebal_display = "{0:.3%}".format(pct_aura_vebal_controlled)
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_display = "{0:.3%}".format(pct_badger_vebal_controlled)
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_badger_vebal_acive_display = "{0:.3%}".format(pct_badger_vebal_controlled_active)
data = [[pct_aura_vebal_display, pct_badger_vebal_display, pct_badger_vebal_acive_display]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,25.450%,0.864%,1.021%


In [36]:
# caps & budger may vary along the quarters modify accordingly
GAUGE_CAP = .02
MAX_BRIBE = 16_000

vlAURA_req = total_vp_badgerdao/(pct_badger_vebal_controlled_active/GAUGE_CAP)
cost_of_max_cap = vlAURA_req * cost_per_vote_after_fee
badger_to_bribe_balancer = cost_of_max_cap / badger_price
bribe_reclycing = badger_to_bribe_balancer * (total_vp_badgerdao/vlAURA_req)
data = [[vlAURA_req, "${:.4f}".format(cost_of_max_cap), badger_to_bribe_balancer, bribe_reclycing]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes = pd.DataFrame(data, columns=headers)
df_bribes

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,536779.36656,$35438.1794,9709.090251,4958.407627


In [37]:
# amount of bribe based on allowance free for bribing elsewhere
bribe_overflow = MAX_BRIBE - badger_to_bribe_balancer
data = [[bribe_overflow]]
headers = ["Badger Bribe Overflow"]
df_bribes_overflow = pd.DataFrame(data, columns=headers)
df_bribes_overflow


,Badger Bribe Overflow
0,6290.909749


In [38]:
# treasury earnings bi-weekly expectation post-bribe
BALANCER_SUBGRAPH = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2"
TVL_QUERY = """
        query($pool_id: String) {
          pool(id: $pool_id) {
            totalLiquidity
          }
        }
        """
POOL_ID_BADGER = "0xb460daa847c45f1c4a41cb05bfb3b51c92e41b36000200000000000000000194"
response_tvl_badger = requests.post(
    BALANCER_SUBGRAPH,
    json={"query": TVL_QUERY, "variables": {"pool_id": POOL_ID_BADGER}},
).json()
tvl_badger = float(response_tvl_badger["data"]["pool"]["totalLiquidity"])
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * GAUGE_CAP
real_bribe_aura_expense = (badger_to_bribe_balancer - bribe_reclycing) * badger_price
treasury_roi = (pool_earnings*pool_capture) / real_bribe_aura_expense
data = [["${:.4f}".format(pool_earnings), "${:.4f}".format(tvl_badger), "{0:.3%}".format(pool_capture), "${:.4f}".format(pool_earnings*pool_capture), "${:.4f}".format(real_bribe_aura_expense), "{0:.3%}".format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "Badger Pool TVL", "Pool Capture", "Treasury yield earned (USD) - biweekly", "Bribe Aura Real Cost", "Treasury ROI - biweekly"]
df_earnings = pd.DataFrame(data, columns=headers)
df_earnings

,Pool yield (USD) - biweekly,Badger Pool TVL,Pool Capture,Treasury yield earned (USD) - biweekly,Bribe Aura Real Cost,Treasury ROI - biweekly
0,$53048.2200,$7537823.9866,38.898%,$20634.7100,$17339.9916,119.001%
